<center>
<img src="https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/img/this_is_data_anal.png" width="200" height="200"><br>
</center>

## 4.2 강남역 맛집 리뷰로 알아보는 감성 분류

분류 모델의 가장 대표적인 활용 방법 중 하나는 바로 감성 분류이다. 감성 분류란 문서를 긍정의견 또는 부정의견으로 나누어 분류하는 것이다. 이번엔 강남역 맛집 리뷰 데이터를 활용하여 감성 분류를 수행해보자.

### step.1 크롤링: 네이버 플레이스 리뷰 크롤링

첫 번째 단계에서는 크롤링을 이용하여 감성 분류에 필요한 데이터를 직접 수집하자. 이번에 크롤링할 데이터는 네이버 플레이스의 음식점 랭킹 정보이다. 네이버 플레이스 API는 별도의 라이브러리 설치나 개발자 등록 없이 source_url에 검색 규칙을 추가하는 것만으로도 크롤링을 적용할 수 있다. 아래의 코드와 같이 start, display, query, sortingOrder 파라미터로 검색 규칙을 추가하자.

- start, display : 검색 결과를 얼마만큼 보여줄지에 관련된 파라미터
- query : 검색하고 싶은 장소나 음식점에 대한 검색어
- sortingOrder : 어떤 방식으로 검색 결과를 정렬할지에 대한 파라미터

검색 규칙을 추가한 url_concat에 request.get 함수를 적용하면 json 형태의 검색 결과 데이터를 얻을 수 있다. 그리고 이를 jason.loads 함수로 변환하면 파이썬의 dictionary와 동일한 형태로 사용할 수 있다.

In [1]:
# -*- coding: utf-8-*-
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

